In [137]:
import pandas as pd
import numpy as np

#Data

The dataset can be found at:
*   https://www.kaggle.com/zygmunt/goodbooks-10k

OR
*   https://github.com/zygmuntz/goodbooks-10k



In [138]:
books_url = "https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv"
ratings_url = "https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv"

r = pd.read_csv(ratings_url)
b = pd.read_csv(books_url)

In [139]:
r.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [140]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [141]:
len(r)

5976479

In [142]:
len(r.user_id.unique())

53424

In [143]:
len(r.book_id.unique())

10000

In [144]:
b.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [145]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    10000 non-null  int64  
 1   goodreads_book_id          10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [146]:

len(b)

10000

In [147]:
book_tags_url = "https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/book_tags.csv"
tags_url = "https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/tags.csv"

book_tags_data = pd.read_csv(book_tags_url)
tags_data = pd.read_csv(tags_url)

In [148]:
book_tags_data.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [149]:
len(book_tags_data)

999912

In [150]:
book_tags_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999912 entries, 0 to 999911
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   goodreads_book_id  999912 non-null  int64
 1   tag_id             999912 non-null  int64
 2   count              999912 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


In [151]:
tags_data.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


DROPPING UNNECESSARY DATA:

In [152]:
b = b.drop(columns=['best_book_id', 'work_id', 'isbn', 'isbn13', 'title','work_ratings_count', 'work_text_reviews_count', 
                    'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url','small_image_url'])
b = b.dropna() 

DROPPING DUPLICATES:

In [153]:
r = r.sort_values("user_id")
r.drop_duplicates(subset =["user_id", "book_id"], keep = False, inplace = True)
b.drop_duplicates(subset='original_title',keep=False,inplace=True)
tags_data.drop_duplicates(subset='tag_id',keep=False,inplace=True)
book_tags_data.drop_duplicates(subset=['tag_id', 'goodreads_book_id'],keep=False,inplace=True)

In [154]:
b.head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count
0,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653
1,2,3,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479
3,4,2657,487,Harper Lee,1960.0,To Kill a Mockingbird,eng,4.25,3198671
4,5,4671,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664
5,6,11870085,226,John Green,2012.0,The Fault in Our Stars,eng,4.26,2346404


#Content Based Recommendation

Content here refers to the content or attributes of the products you like. So, the idea in content-based filtering is to tag products using certain keywords, understand what the user likes, look up those keywords in the database and recommend different products with the same attributes.

In [155]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [156]:
content_data = b[['original_title','authors','average_rating']]
content_data = content_data.astype(str)

In [157]:
content_data['content'] = content_data['original_title'] + ' ' + content_data['authors'] + ' ' + content_data['average_rating']

A NEW DATASET, AND A NEW SERIES TO STORE INDEX NO. AND TITLE:

In [158]:
content_data = content_data.reset_index()
indices = pd.Series(content_data.index, index=content_data['original_title'])

In [171]:
content_data

,index,original_title,authors,average_rating,content
0,0,The Hunger Games,Suzanne Collins,4.34,The Hunger Games Suzanne Collins 4.34
1,1,Harry Potter and the Philosopher's Stone,"J.K. Rowling, Mary GrandPré",4.44,Harry Potter and the Philosopher's Stone J.K. ...
2,3,To Kill a Mockingbird,Harper Lee,4.25,To Kill a Mockingbird Harper Lee 4.25
3,4,The Great Gatsby,F. Scott Fitzgerald,3.89,The Great Gatsby F. Scott Fitzgerald 3.89
4,5,The Fault in Our Stars,John Green,4.26,The Fault in Our Stars John Green 4.26
...,...,...,...,...,...
8170,9994,"Billy Budd, Sailor",Herman Melville,3.09,"Billy Budd, Sailor Herman Melville 3.09"
8171,9995,Bayou Moon,Ilona Andrews,4.09,Bayou Moon Ilona Andrews 4.09
8172,9996,Means of Ascent,Robert A. Caro,4.25,Means of Ascent Robert A. Caro 4.25
8173,9997,The Mauritius Command,Patrick O'Brian,4.35,The Mauritius Command Patrick O'Brian 4.35


In [160]:
indices

original_title
The Hunger Games                                                                                0
Harry Potter and the Philosopher's Stone                                                        1
To Kill a Mockingbird                                                                           2
The Great Gatsby                                                                                3
The Fault in Our Stars                                                                          4
                                                                                             ... 
Billy Budd, Sailor                                                                           8170
Bayou Moon                                                                                   8171
Means of Ascent                                                                              8172
The Mauritius Command                                                                        8173
Cinde

#CREATING A TF-IDF MATRIX BASED ON "AUTHORS":

In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general.

In [161]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(content_data['authors'])
tfidf_matrix.shape

(8175, 5484)

COMPUTING COSINE SIMILARITIES:

In [162]:
cosine_sim_author = linear_kernel(tfidf_matrix, tfidf_matrix)

In [163]:
cosine_sim_author

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

**FUNCTION THAT WILL RETURN THE TOP 10 BOOKS WHICH ARE WRITTEN BY AUTHOR WITH SIMILAR "average_rating" AS THE AUTHOR OF BOOK ASSIGNED TO "title" VARIABLE:**

In [164]:
def get_recommendations_books_author(title, cosine_sim = cosine_sim_author):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim_author[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:11]
  book_indices = [i[0] for i in sim_scores]
  return list(content_data['original_title'].iloc[book_indices])

In [165]:
def book_shows(books):
  print("Recommended Books:\n")
  for book in books:
    print(book)

EXAMPLE:

In [166]:
books = get_recommendations_books('The Mauritius Command', cosine_sim_author)
book_shows(books)

Recommended Books:

Post Captain
H.M.S. Surprise 
The Mauritius Command
The Knife of Never Letting Go
The Ask and the Answer
Monsters of Men
More Than This
The Rest of Us Just Live Here
The Name of the Wind
The Wise Man's Fear


#CREATING COUNT_VECTORIZER MATRIX, BASED ON "CONTENT"(i.e multiple attributes):

In order to use textual data for predictive modeling, the text must be parsed to remove certain words – this process is called tokenization. These words need to then be encoded as integers, or floating-point values, for use as inputs in machine learning algorithms. This process is called feature extraction (or vectorization).

Scikit-learn’s CountVectorizer is used to convert a collection of text documents to a vector of term/token counts. It also enables the ​pre-processing of text data prior to generating the vector representation. This functionality makes it a highly flexible feature representation module for text.

In [174]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(content_data['content'])
count_matrix.shape

(8175, 13024)

COMPUTING COSINE SIMILARITIES:

In [168]:
cosine_sim_content = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim_content)

[[1.  0.  0.  ... 0.  0.  0. ]
 [0.  1.  0.  ... 0.  0.  0. ]
 [0.  0.  1.  ... 0.2 0.  0. ]
 ...
 [0.  0.  0.2 ... 1.  0.  0. ]
 [0.  0.  0.  ... 0.  1.  0. ]
 [0.  0.  0.  ... 0.  0.  1. ]]


**FUNCTION THAT WILL RETURN THE TOP 10 BOOKS WHICH HAVE SIMILAR "content" AS THE BOOK ASSIGNED TO "title" VARIABLE:**

In [169]:
def get_recommendations_content(title, cosine_sim=cosine_sim_content):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim_content[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:11]
  book_indices = [i[0] for i in sim_scores]
  return list(content_data['original_title'].iloc[book_indices])

EXAMPLE:

In [172]:
books = get_recommendations_content("Bayou Moon", cosine_sim_content)
book_shows(books)

Recommended Books:

On the Edge
Clean Sweep
Burn For Me
Magic Bites
Magic Strikes
Magic Bleeds
Magic Burns
Magic Slays
Emily of New Moon
Magic Rises
